In [1]:
%config IPCompleter.greedy=True

In [2]:
# Importing library
import pandas as pd  

In [3]:
# Reading CSV inputs
marks = pd.read_csv("marks.csv")
test = pd.read_csv("tests.csv", header=0, names=['test_id', 'course_id', 'weight'])
student = pd.read_csv("students.csv", names=['student_id', "student_name"], header=0)
course = pd.read_csv("courses.csv", header=0, names=['course_id', 'course_name', 'teacher'])

In [4]:
# Making a common CSV
marks_merge = marks.merge(student, how='left', on='student_id')
test_merge = test.merge(course, how='left', on='course_id')
final_merge = pd.merge(marks_merge, test_merge, on='test_id')

In [5]:
# Checking the CSV column names
columnTitle = ['student_id', 'test_id', 'mark', 'student_name', 'course_id', 'weight', 'course_name', 'teacher']
final_merge = final_merge.reindex(columns=columnTitle).sort_values(by=['student_id', 'course_id', 'test_id']).reset_index(drop=True)

In [6]:
final_merge

,student_id,test_id,mark,student_name,course_id,weight,course_name,teacher
0,1,1,78,A,1,10,Biology,Mr. D
1,1,2,87,A,1,40,Biology,Mr. D
2,1,3,95,A,1,50,Biology,Mr. D
3,1,4,32,A,2,40,History,Mrs. P
4,1,5,65,A,2,60,History,Mrs. P
5,1,6,78,A,3,90,Math,Mrs. C
6,1,7,40,A,3,10,Math,Mrs. C
7,2,1,78,B,1,10,Biology,Mr. D
8,2,2,87,B,1,40,Biology,Mr. D
9,2,3,15,B,1,50,Biology,Mr. D


In [7]:
# Calculating the weighted marks
final_merge['weighted_marks'] = final_merge['mark'] * final_merge['weight'] / 100

In [8]:
# Number of students we have, their ID
student_list = list(set(final_merge['student_id']))

In [9]:
def write_to_file(student_name, student_id, course_name, teacher, marks, average):
    file = open("report_card.txt", "a")

    file.write("Student Id: %d, name: %s\n" % (student_id[0], student_name[0]))
    file.write("Total Average:\t\t%.2f%%\n\n" % (average))
    for i in range(len(course_name)):
        file.write("\tCourse: %s, Teacher: %s\n" % (course_name[i], teacher[i]))
        file.write("\tFinal Grade:\t%.2f%%" % (marks[i]))
        file.write("\n")
    file.write('\n\n')
    file.close()

In [10]:
def extract_info(df1):
    student_name, student_id = list(set(df1['student_name'])), list(set(df1['student_id']))
    course_name, teacher = list(df1['course_name']), list(df1['teacher'])
    marks = list(df1['weighted_marks'])
    sum_of_marks = 0
    for i in marks:
        sum_of_marks += i
    average = sum_of_marks/len(marks)
    write_to_file(student_name, student_id, course_name, teacher, marks, average)

In [11]:
def drop_extra_cols(df):
    df1 = df.groupby('course_id').agg({'student_id' : lambda student_id: student_id.unique(),
                                    'student_name' : lambda student_name: student_name.unique(),
                                    'course_name' : lambda course_name: course_name.unique(),
                                    'teacher' : lambda teacher: teacher.unique(),
                                    'weighted_marks': lambda weighted_marks: weighted_marks.sum(),})
    extract_info(df1)

In [12]:
for i in student_list:
    df = final_merge.loc[(final_merge['student_id']==i)]
    drop_extra_cols(df)